# Projecte de GDSA
## Batch size i inicialització de pesos.
Equip 3:
Nil Torrents, Adrián Rodríguez, Eugeni Valentí, Roger Vallejo

In [ ]:
# Random seed 123 from numpy
import numpy as np
np.random.seed(123)
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from google.colab import files
import torch.optim as optim
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from timeit import default_timer as timer

## Definició d'hiperparàmetres

In [ ]:
# Let's define some hyper-parameters
hparams = {
    'batch_size':64,
    'num_epochs':10,
    'test_batch_size':64,
    'hidden_size':128,
    'num_classes':10,
    'num_inputs':784,
    'learning_rate':1e-3,
    'log_interval':100,
}

hparams_batchSize1000 = {
    'batch_size':1000,
    'num_epochs':10,
    'test_batch_size':1000,
    'hidden_size':128,
    'num_classes':10,
    'num_inputs':784,
    'learning_rate':1e-3,
    'log_interval':100,
}

hparams_batchSize500 = {
    'batch_size':500,
    'num_epochs':10,
    'test_batch_size':500,
    'hidden_size':128,
    'num_classes':10,
    'num_inputs':784,
    'learning_rate':1e-3,
    'log_interval':100,
}

hparams_batchSize1 = {
    'batch_size':1,
    'num_epochs':1,
    'test_batch_size':1,
    'hidden_size':128,
    'num_classes':10,
    'num_inputs':784,
    'learning_rate':1e-3,
    'log_interval':100,
}

# we select to work on GPU if it is available in the machine, otherwise
# will run on CPU
hparams['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
hparams_batchSize1000['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
hparams_batchSize1['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

## Definició de Dataset i DataLoader


In [ ]:
mnist_trainset = datasets.MNIST('data', train=True, download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))
mnist_testset = datasets.MNIST('data', train=False, 
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))

train_loader = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=hparams['batch_size'], 
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    mnist_testset,
    batch_size=hparams['test_batch_size'], 
    shuffle=False)

train_loader_batchSize1000 = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=hparams_batchSize1000['batch_size'], 
    shuffle=True)

test_loader_batchSize1000 = torch.utils.data.DataLoader(
    mnist_testset,
    batch_size=hparams_batchSize1000['test_batch_size'], 
    shuffle=False)

train_loader_batchSize1 = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=hparams_batchSize1['batch_size'], 
    shuffle=True)

test_loader_batchSize1 = torch.utils.data.DataLoader(
    mnist_testset,
    batch_size=hparams_batchSize1['test_batch_size'], 
    shuffle=False)

train_loader_batchSize500 = torch.utils.data.DataLoader(
    mnist_trainset,
    batch_size=hparams_batchSize500['batch_size'], 
    shuffle=True)

test_loader_batchSize500 = torch.utils.data.DataLoader(
    mnist_testset,
    batch_size=hparams_batchSize500['test_batch_size'], 
    shuffle=False)


# Definició de les arquitectures neuronals

In [ ]:
import torch

def describe_tensor(tensor, name=''):
  # Helper function to explore the attributes of a tensor object
  print('-' * 30)
  print('Name: ', name)
  print('-' * 30)
  print('data : ', tensor.data)
  print('requires_grad : ', tensor.requires_grad)
  print('grad: ', tensor.grad)
  print('grad_fn: ', tensor.grad_fn)
  print('is_leaf: ', tensor.is_leaf)
  print('=' * 30)

In [ ]:
# Add a (1) nn.Linear hidden layer with 128 neurons, (2) a nn.ReLU, (3) the output nn.Linear and,
# the (4) output nn.LogSoftmax
# NOTE: Consider the 'num_inputs', 'hidden_size', and 'num_classes' parameters 
# defined above as hyper-params

#TODO
network=nn.Sequential(
    nn.Linear(hparams['num_inputs'], hparams['hidden_size'], True),
    nn.ReLU(inplace=True),
    nn.Linear(hparams['hidden_size'], hparams['num_classes'], True),
    nn.LogSoftmax()
)
network.to(hparams['device'])

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU(inplace=True)
  (2): Linear(in_features=128, out_features=10, bias=True)
  (3): LogSoftmax(dim=None)
)

Inicialitzada a zeros_

In [ ]:
import torch.nn as nn


class MyNet(nn.Module):
  
  def __init__(self):
    super().__init__() # must call the superclass init first
    # First fully-connected layer (3 inputs, 20 hidden neurons)
    # self.fc1 = nn.Linear(3, 20)
    
    self.fc1 = nn.Linear(hparams['num_inputs'], hparams['hidden_size'])
    nn.init.zeros_(self.fc1.weight)
    # First hidden activation
    self.act1 = nn.ReLU(inplace=True)
    # Second fully-connected layer (20 hidden neurons, 3 outputs)
    self.fc2 = nn.Linear(hparams['hidden_size'], hparams['num_classes'])
    # No activation as we make it a linear output
    self.act2 = nn.Softmax(dim=None)
    
  def forward(self, x):
    # activation of first layer is Tanh(FC1(x))
    h1 = self.act1(self.fc1(x))
    # output activation
    y = self.fc2(h1)
    return y

In [ ]:
netZeros = MyNet() #ens diu q tenim a dins d la xarxa
netZeros.to(hparams['device'])
print(netZeros)

MyNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (act1): ReLU(inplace=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (act2): Softmax(dim=None)
)


In [ ]:
describe_tensor(netZeros.fc1.weight, 'FC1 weight') #Nomes layer 1. podem mirar els w. Començen sent w i b aleatoris.

------------------------------
Name:  FC1 weight
------------------------------
data :  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')
requires_grad :  True
grad:  None
grad_fn:  None
is_leaf:  True


Inicialitzada a Xavier


In [ ]:
import torch.nn as nn


class MyNet(nn.Module):
  
  def __init__(self):
    super().__init__() # must call the superclass init first
    # First fully-connected layer (3 inputs, 20 hidden neurons)
    # self.fc1 = nn.Linear(3, 20)
    
    self.fc1 = nn.Linear(hparams['num_inputs'], hparams['hidden_size'])
    nn.init.xavier_normal_(self.fc1.weight)
    # First hidden activation
    self.act1 = nn.Tanh()
    # Second fully-connected layer (20 hidden neurons, 3 outputs)
    self.fc2 = nn.Linear(hparams['hidden_size'], hparams['num_classes'])
    # No activation as we make it a linear output
    self.act2 = nn.Softmax(dim=None)
    
  def forward(self, x):
    # activation of first layer is Tanh(FC1(x))
    h1 = self.act1(self.fc1(x))
    # output activation
    y = self.fc2(h1)
    return y

In [ ]:
netXavier = MyNet() #ens diu q tenim a dins d la xarxa
# Send the network to the proper device (CPU or GPU)
netXavier.to(hparams['device'])
print(netXavier)

MyNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (act1): Tanh()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (act2): Softmax(dim=None)
)


In [ ]:
describe_tensor(netXavier.to(hparams['device'])
.fc1.weight, 'FC1 weight') #Nomes layer 1. podem mirar els w. Començen sent w i b aleatoris.

------------------------------
Name:  FC1 weight
------------------------------
data :  tensor([[-0.0366,  0.0228,  0.0238,  ..., -0.0249, -0.0505, -0.0063],
        [-0.0691,  0.0342, -0.0060,  ..., -0.0400,  0.0604,  0.0872],
        [ 0.0350, -0.0481,  0.0140,  ...,  0.0256,  0.0626, -0.0409],
        ...,
        [-0.0133,  0.0751, -0.0187,  ...,  0.0126,  0.0745,  0.0653],
        [-0.0463,  0.0692,  0.0974,  ..., -0.1017,  0.0128, -0.0070],
        [ 0.0970, -0.0049, -0.0868,  ..., -0.0373,  0.0303,  0.0212]],
       device='cuda:0')
requires_grad :  True
grad:  None
grad_fn:  None
is_leaf:  True


batchSize=500, inicialització Zeros

In [ ]:
import torch.nn as nn


class MyNet(nn.Module):
  
  def __init__(self):
    super().__init__() # must call the superclass init first
    # First fully-connected layer (3 inputs, 20 hidden neurons)
    # self.fc1 = nn.Linear(3, 20)
    
    self.fc1 = nn.Linear(hparams['num_inputs'], hparams['hidden_size'])
    nn.init.zeros_(self.fc1.weight)
    # First hidden activation
    self.act1 = nn.ReLU(inplace=True)
    # Second fully-connected layer (20 hidden neurons, 3 outputs)
    self.fc2 = nn.Linear(hparams['hidden_size'], hparams['num_classes'])
    # No activation as we make it a linear output
    self.act2 = nn.Softmax(dim=None)
    
  def forward(self, x):
    # activation of first layer is Tanh(FC1(x))
    h1 = self.act1(self.fc1(x))
    # output activation
    y = self.fc2(h1)
    return y

In [ ]:
netZeros_500 = MyNet() #ens diu q tenim a dins d la xarxa
# Send the network to the proper device (CPU or GPU)
netZeros_500.to(hparams['device'])
print(netZeros_500)

MyNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (act1): ReLU(inplace=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (act2): Softmax(dim=None)
)


batchSize=500, inicialització Xavier

In [ ]:
import torch.nn as nn


class MyNet(nn.Module):
  
  def __init__(self):
    super().__init__() # must call the superclass init first
    # First fully-connected layer (3 inputs, 20 hidden neurons)
    # self.fc1 = nn.Linear(3, 20)
    
    self.fc1 = nn.Linear(hparams['num_inputs'], hparams['hidden_size'])
    nn.init.xavier_normal_(self.fc1.weight)
    # First hidden activation
    self.act1 = nn.Tanh()
    # Second fully-connected layer (20 hidden neurons, 3 outputs)
    self.fc2 = nn.Linear(hparams['hidden_size'], hparams['num_classes'])
    # No activation as we make it a linear output
    self.act2 = nn.Softmax(dim=None)
    
  def forward(self, x):
    # activation of first layer is Tanh(FC1(x))
    h1 = self.act1(self.fc1(x))
    # output activation
    y = self.fc2(h1)
    return y

In [ ]:
netXavier_500 = MyNet() #ens diu q tenim a dins d la xarxa
# Send the network to the proper device (CPU or GPU)
netXavier_500.to(hparams['device'])
print(netXavier_500)

MyNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (act1): Tanh()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (act2): Softmax(dim=None)
)


In [ ]:
def get_nn_nparams(net):
  """ https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6 """
  pp=0
  for p in list(net.parameters()):
      nn=1
      for s in list(p.size()):
          nn = nn*s
      pp += nn
  return pp
  
print(network)
print('Num params: ', get_nn_nparams(network))

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU(inplace=True)
  (2): Linear(in_features=128, out_features=10, bias=True)
  (3): LogSoftmax(dim=None)
)
Num params:  101770


In [ ]:
# We can access all the parameters of our network with the .parameters() function, that returns an iterable
# over all tunnable params we created.
params = list(netZeros.parameters())
for p in params:
  print(p.shape)
print('You should see two matrices (weights, OUTxIN) and two vectors (biases, OUT). Each pair of weight (W) and bias (b) comes from a fully connected layer.')

torch.Size([128, 784])
torch.Size([128])
torch.Size([10, 128])
torch.Size([10])
You should see two matrices (weights, OUTxIN) and two vectors (biases, OUT). Each pair of weight (W) and bias (b) comes from a fully connected layer.


# Optimitzador

In [ ]:
# Import optmizer 
import torch.optim as optim

# Import a functional API for the loss function (use this one !!)
import torch.nn.functional as F

# Optimizer: RMS Prop (use the hparams['learning_rate'] previously defined)
# https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop
optimizer=optim.RMSprop(network.parameters(), hparams['learning_rate'])
optimizerZeros=optim.RMSprop(netZeros.parameters(), hparams['learning_rate'])
optimizerXavier=optim.RMSprop(netXavier.parameters(), hparams['learning_rate'])
optimizerZeros_500=optim.RMSprop(netZeros_500.parameters(), hparams['learning_rate'])
optimizerXavier_500=optim.RMSprop(netXavier_500.parameters(), hparams['learning_rate'])




# Negative Log Likelihood (NLL) Loss criterion from the functional API
# https://pytorch.org/docs/stable/nn.functional.html#nll-loss
criterion=F.nll_loss

# Define the Accuracy metric in the function below by:
  # (1) obtain the maximum for each predicted element in the batch to get the class (it is the maximum index of the num_classes array per batch sample) (look at torch.argmax in the PyTorch documentation)
  # (2) compare the predicted class index with the index in its corresponding neighbor within label_batch 
  # (3) sum up the number of affirmative comparisons and return the summation
def correct_predictions(predicted_batch, label_batch):
  pred = predicted_batch.argmax(dim=1, keepdim=True) # get the index of the max log-probability
  acum = pred.eq(label_batch.view_as(pred)).sum().item()
  return acum

# Train epochs

batchSize=64, inicialització random

In [ ]:
def train_epoch(train_loader, network, optimizer, criterion, hparams):

  # Activate the train=True flag inside the model
  network.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = network(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizer.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

batchSize=500, inicialització random

In [ ]:
def train_epoch_500(train_loader, network, optimizer, criterion, hparams):

  # Activate the train=True flag inside the model
  network.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = network(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizer.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

Entrenem batchSize=64 amb netZeros!

In [ ]:
def train_epoch_netZeros(train_loader, netZeros, optimizerZeros, criterion, hparams):

  # Activate the train=True flag inside the model
  netZeros.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizerZeros.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = netZeros(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizerZeros.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

Entrenem batchSize64 amb Xavier

In [ ]:
def train_epoch_xavier(train_loader, netXavier, optimizerXavier, criterion, hparams):

  # Activate the train=True flag inside the model
  netXavier.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizerXavier.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = netXavier(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizerXavier.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

batchSize=1000

In [ ]:
def train_epoch_batchSize1000(train_loader_batchSize1000, network, optimizer, criterion, hparams_batchSize1000):

  # Activate the train=True flag inside the model
  network.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader_batchSize1000):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = network(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizer.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader_batchSize1000.dataset),
              100. * batch_idx / len(train_loader_batchSize1000), loss.item()))
  return avg_loss

batchSize=1

In [ ]:
def train_epoch_batchSize1(train_loader_batchSize1, network, optimizer, criterion, hparams_batchSize1):

  # Activate the train=True flag inside the model
  network.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader_batchSize1):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = network(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizer.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader_batchSize1.dataset),
              100. * batch_idx / len(train_loader_batchSize1), loss.item()))
  return avg_loss

batchSize=500, Zeros

In [ ]:
def train_epoch_netZeros_500(train_loader, netZeros_500, optimizerZeros_500, criterion, hparams):

  # Activate the train=True flag inside the model
  netZeros_500.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizerZeros_500.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = netZeros_500(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizerZeros_500.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

batchSize=500, Xavier

In [ ]:
def train_epoch_xavier_500(train_loader, netXavier_500, optimizerXavier_500, criterion, hparams):

  # Activate the train=True flag inside the model
  netXavier_500.train()
  
  device = hparams['device']
  avg_loss = None
  avg_weight = 0.1

  # For each batch
  for batch_idx, (data, target) in enumerate(train_loader):
      data, target = data.to(device), target.to(device)
      optimizerXavier_500.zero_grad()
      # TO DO
      data = data.view(data.shape[0], -1)
      output = netXavier_500(data)
      loss = criterion(output, target)
      loss.backward()
      if avg_loss:
        avg_loss = avg_weight * loss.item() + (1 - avg_weight) * avg_loss
      else:
        avg_loss = loss.item()
      optimizerXavier.step()
      if batch_idx % hparams['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
  return avg_loss

##Tests

batchSize=64, inicialització random

In [ ]:
def test_epoch(test_loader, network, hparams):

    # Dectivate the train=True flag inside the model
    network.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = network(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

batchSize=500, inicialització random

In [ ]:
def test_epoch_500(test_loader, network, hparams):

    # Dectivate the train=True flag inside the model
    network.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = network(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

netZeros!

In [ ]:
def test_epoch_netZeros(test_loader, netZeros, hparams):

    # Dectivate the train=True flag inside the model
    netZeros.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = netZeros(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

netZeros, batchSize=500!

In [ ]:
def test_epoch_netZeros_500(test_loader, netZeros_500, hparams):

    # Dectivate the train=True flag inside the model
    netZeros_500.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = netZeros_500(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

xavier!

In [ ]:
def test_epoch_xavier(test_loader, netXavier, hparams):

    # Dectivate the train=True flag inside the model
    netXavier.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = netXavier(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

xavier, batchSize=500!

In [ ]:
def test_epoch_xavier_500(test_loader, netXavier_500, hparams):

    # Dectivate the train=True flag inside the model
    netXavier_500.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = netXavier_500(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader.dataset)
    test_acc = 100. * acc / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader.dataset), test_acc,
        ))
    return test_loss, test_acc

In [ ]:
def test_epoch_batchSize1000(test_loader_batchSize1000, network, hparams_batchSize1000):

    # Dectivate the train=True flag inside the model
    network.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = network(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader_batchSize1000.dataset)
    test_acc = 100. * acc / len(test_loader_batchSize1000.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader_batchSize1000.dataset), test_acc,
        ))
    return test_loss, test_acc

In [ ]:
def test_epoch_batchSize1(test_loader_batchSize1, network, hparams_batchSize1):

    # Dectivate the train=True flag inside the model
    network.eval()
    
    device = hparams['device']
    test_loss = 0
    acc = 0
    with torch.no_grad():
        for data, target in test_loader:

            # Load data and feed it through the neural network
            data, target = data.to(device), target.to(device)
            data = data.view(data.shape[0], -1)
            output = network(data)

            test_loss += criterion(output, target, reduction='sum').item() # sum up batch loss
            # WARNING: If you are using older Torch versions, the previous call may need to be replaced by
            # test_loss += criterion(output, target, size_average=False).item()

            # compute number of correct predictions in the batch
            acc += correct_predictions(output, target)

    # Average accuracy across all correct predictions batches now
    test_loss /= len(test_loader_batchSize1.dataset)
    test_acc = 100. * acc / len(test_loader_batchSize1.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, acc, len(test_loader_batchSize1.dataset), test_acc,
        ))
    return test_loss, test_acc

# Entrenaments i tests definitius

batchSize = 64, inicialització random

In [ ]:
import time
start_time1 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses = []
test_losses = []
test_accs = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

  # Compute & save the average training loss for the current epoch
  train_loss = train_epoch(train_loader, network, optimizer, criterion, hparams)
  train_losses.append(train_loss)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss,test_accuracy=test_epoch(test_loader, network, hparams)
  test_losses.append(test_loss)
  test_accs.append(test_accuracy)

  temps_batchSize64 = (time.time() - start_time1)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.261841
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.197675
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.364292
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.177752
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.108788
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.377456
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.146492
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.095700
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.336215
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.160645

Test set: Average loss: 0.1352, Accuracy: 9553/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.075977
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.075671
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.027029
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.136358
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.099991
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.147775
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.049865
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.107740
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.128945
T

batchSize = 500, inicialització random

In [ ]:
import time
start_time8 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_500 = []
test_losses_500 = []
test_accs_500 = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

  # Compute & save the average training loss for the current epoch
  train_loss = train_epoch(train_loader, network, optimizer, criterion, hparams)
  train_losses_500.append(train_loss)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss,test_accuracy=test_epoch_500(test_loader, network, hparams)
  test_losses_500.append(test_loss)
  test_accs_500.append(test_accuracy)

  temps_batchSize500 = (time.time() - start_time8)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.001920


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.001901
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.005644
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001749
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.040331
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.000383
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.001324
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.000416
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.020879
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.028290

Test set: Average loss: 0.0958, Accuracy: 9765/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.006064
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.009661
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.000407
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.000724
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.035877
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.035136
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.000137
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.008060
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.012360
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.089

netZeros

In [ ]:
import time
start_time4 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_zeros = []
test_losses_zeros = []
test_accs_zeros = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

  # Compute & save the average training loss for the current epoch
  train_loss = train_epoch_netZeros(train_loader, netZeros, optimizerZeros, criterion, hparams)
  train_losses_zeros.append(train_loss)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss,test_accuracy=test_epoch_netZeros(test_loader, netZeros, hparams)
  test_losses_zeros.append(test_loss)
  test_accs_zeros.append(test_accuracy)

  temps_batchSize64_netZeros = (time.time() - start_time4)

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.034656
Train Epoch: 1 [6400/60000 (11%)]	Loss: -1149.273315
Train Epoch: 1 [12800/60000 (21%)]	Loss: -2736.925293
Train Epoch: 1 [19200/60000 (32%)]	Loss: -4840.976074
Train Epoch: 1 [25600/60000 (43%)]	Loss: -7378.656738
Train Epoch: 1 [32000/60000 (53%)]	Loss: -9253.627930
Train Epoch: 1 [38400/60000 (64%)]	Loss: -13289.453125
Train Epoch: 1 [44800/60000 (75%)]	Loss: -16904.234375
Train Epoch: 1 [51200/60000 (85%)]	Loss: -21156.621094
Train Epoch: 1 [57600/60000 (96%)]	Loss: -24289.083984

Test set: Average loss: -26894.5644, Accuracy: 980/10000 (10%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -25402.150391
Train Epoch: 2 [6400/60000 (11%)]	Loss: -33133.031250
Train Epoch: 2 [12800/60000 (21%)]	Loss: -36489.167969
Train Epoch: 2 [19200/60000 (32%)]	Loss: -42219.699219
Train Epoch: 2 [25600/60000 (43%)]	Loss: -45760.832031
Train Epoch: 2 [32000/60000 (53%)]	Loss: -54262.785156
Train Epoch: 2 [38400/60000 (64%)]	Loss: -59488.988281
Train Epoch: 2 [44800

Zeros, batchSize=500

In [ ]:
import time
start_time6 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_zeros_500 = []
test_losses_zeros_500 = []
test_accs_zeros_500 = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

  # Compute & save the average training loss for the current epoch
  train_loss = train_epoch_netZeros_500(train_loader, netZeros, optimizerZeros, criterion, hparams)
  train_losses_zeros_500.append(train_loss)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss,test_accuracy=test_epoch_netZeros_500(test_loader, netZeros, hparams)
  test_losses_zeros_500.append(test_loss)
  test_accs_zeros_500.append(test_accuracy)

  temps_batchSize64_netZeros_500 = (time.time() - start_time6)

Train Epoch: 1 [0/60000 (0%)]	Loss: -1712501.500000
Train Epoch: 1 [6400/60000 (11%)]	Loss: -1787687.000000
Train Epoch: 1 [12800/60000 (21%)]	Loss: -1783018.125000
Train Epoch: 1 [19200/60000 (32%)]	Loss: -1848339.000000
Train Epoch: 1 [25600/60000 (43%)]	Loss: -1802638.000000
Train Epoch: 1 [32000/60000 (53%)]	Loss: -1821400.875000
Train Epoch: 1 [38400/60000 (64%)]	Loss: -1938707.875000
Train Epoch: 1 [44800/60000 (75%)]	Loss: -2059598.375000
Train Epoch: 1 [51200/60000 (85%)]	Loss: -1989203.250000
Train Epoch: 1 [57600/60000 (96%)]	Loss: -1991740.000000

Test set: Average loss: -2075728.2432, Accuracy: 1135/10000 (11%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -1926811.875000
Train Epoch: 2 [6400/60000 (11%)]	Loss: -2135999.500000
Train Epoch: 2 [12800/60000 (21%)]	Loss: -2058953.250000
Train Epoch: 2 [19200/60000 (32%)]	Loss: -2142917.750000
Train Epoch: 2 [25600/60000 (43%)]	Loss: -2265528.500000
Train Epoch: 2 [32000/60000 (53%)]	Loss: -2144633.500000
Train Epoch: 2 [38400/60000 (64%

xavier

In [ ]:
import time
start_time5 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_xavier = []
test_losses_xavier = []
test_accs_xavier = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

  # Compute & save the average training loss for the current epoch
  train_loss = train_epoch_xavier(train_loader, netXavier, optimizerXavier, criterion, hparams)
  train_losses_xavier.append(train_loss)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss,test_accuracy=test_epoch_xavier(test_loader, netXavier, hparams)
  test_losses_xavier.append(test_loss)
  test_accs_xavier.append(test_accuracy)

  temps_batchSize64_xavier = (time.time() - start_time5)

Train Epoch: 1 [0/60000 (0%)]	Loss: -0.074014
Train Epoch: 1 [6400/60000 (11%)]	Loss: -25.195967
Train Epoch: 1 [12800/60000 (21%)]	Loss: -38.931900
Train Epoch: 1 [19200/60000 (32%)]	Loss: -50.573986
Train Epoch: 1 [25600/60000 (43%)]	Loss: -62.731064
Train Epoch: 1 [32000/60000 (53%)]	Loss: -74.540855
Train Epoch: 1 [38400/60000 (64%)]	Loss: -85.354576
Train Epoch: 1 [44800/60000 (75%)]	Loss: -98.695915
Train Epoch: 1 [51200/60000 (85%)]	Loss: -109.813400
Train Epoch: 1 [57600/60000 (96%)]	Loss: -119.050858

Test set: Average loss: -125.6080, Accuracy: 8827/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -125.132309
Train Epoch: 2 [6400/60000 (11%)]	Loss: -137.197037
Train Epoch: 2 [12800/60000 (21%)]	Loss: -148.016693
Train Epoch: 2 [19200/60000 (32%)]	Loss: -160.532120
Train Epoch: 2 [25600/60000 (43%)]	Loss: -171.947357
Train Epoch: 2 [32000/60000 (53%)]	Loss: -185.571426
Train Epoch: 2 [38400/60000 (64%)]	Loss: -197.371674
Train Epoch: 2 [44800/60000 (75%)]	Loss: -208.458496
Tra

Xavier, batchSize=500

In [ ]:
import time
start_time7 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_xavier_500 = []
test_losses_xavier_500 = []
test_accs_xavier_500 = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

  # Compute & save the average training loss for the current epoch
  train_loss = train_epoch_xavier_500(train_loader, netXavier, optimizerXavier, criterion, hparams)
  train_losses_xavier_500.append(train_loss)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss,test_accuracy=test_epoch_xavier(test_loader, netXavier, hparams)
  test_losses_xavier_500.append(test_loss)
  test_accs_xavier_500.append(test_accuracy)

  temps_batchSize64_xavier_500 = (time.time() - start_time7)

Train Epoch: 1 [0/60000 (0%)]	Loss: -1100.501831
Train Epoch: 1 [6400/60000 (11%)]	Loss: -1134.852295
Train Epoch: 1 [12800/60000 (21%)]	Loss: -1130.218872
Train Epoch: 1 [19200/60000 (32%)]	Loss: -1150.363770
Train Epoch: 1 [25600/60000 (43%)]	Loss: -1176.216431
Train Epoch: 1 [32000/60000 (53%)]	Loss: -1168.515137
Train Epoch: 1 [38400/60000 (64%)]	Loss: -1175.393311
Train Epoch: 1 [44800/60000 (75%)]	Loss: -1196.521118
Train Epoch: 1 [51200/60000 (85%)]	Loss: -1220.320435
Train Epoch: 1 [57600/60000 (96%)]	Loss: -1222.142944

Test set: Average loss: -1220.3821, Accuracy: 9096/10000 (91%)

Train Epoch: 2 [0/60000 (0%)]	Loss: -1227.137329
Train Epoch: 2 [6400/60000 (11%)]	Loss: -1247.003662
Train Epoch: 2 [12800/60000 (21%)]	Loss: -1247.610352
Train Epoch: 2 [19200/60000 (32%)]	Loss: -1252.705811
Train Epoch: 2 [25600/60000 (43%)]	Loss: -1252.266113
Train Epoch: 2 [32000/60000 (53%)]	Loss: -1296.231689
Train Epoch: 2 [38400/60000 (64%)]	Loss: -1284.385132
Train Epoch: 2 [44800/60000 (

batchSize=1000

In [ ]:
start_time2 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_batchSize1000 = []
test_losses_batchSize1000 = []
test_accs_batchSize1000 = []

# For each epoch
for epoch in range(1, hparams['num_epochs'] + 1):

# Compute & save the average training loss for the current epoch
  train_loss_batchSize1000 = train_epoch_batchSize1000(train_loader_batchSize1000, network, optimizer, criterion, hparams_batchSize1000)
  train_losses_batchSize1000.append(train_loss_batchSize1000)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss_batchSize1000,test_accuracy_batchSize1000=test_epoch_batchSize1000(test_loader_batchSize1000, network, hparams_batchSize1000)
  test_losses_batchSize1000.append(test_loss_batchSize1000)
  test_accs_batchSize1000.append(test_accuracy_batchSize1000)

  temps_batchSize1000 = (time.time() - start_time2)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.013345


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)



Test set: Average loss: 0.0997, Accuracy: 9816/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.001212

Test set: Average loss: 0.0982, Accuracy: 9823/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.000804

Test set: Average loss: 0.0982, Accuracy: 9818/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.000498

Test set: Average loss: 0.0981, Accuracy: 9816/10000 (98%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.000476

Test set: Average loss: 0.0975, Accuracy: 9819/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.000282

Test set: Average loss: 0.0979, Accuracy: 9816/10000 (98%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.000220

Test set: Average loss: 0.0979, Accuracy: 9815/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.000241

Test set: Average loss: 0.0979, Accuracy: 9815/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.000118

Test set: Average loss: 0.0986, Accuracy: 9817/10000 (98%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.000161

Test set: Average loss: 0.0989, Accuracy: 98

In [ ]:
start_time3 = time.time()

# Init lists to save the evolution of the training & test losses/accuracy.
train_losses_batchSize1 = []
test_losses_batchSize1 = []
test_accs_batchSize1 = []

# For each epoch
for epoch in range(1, hparams_batchSize1['num_epochs'] + 1):

# Compute & save the average training loss for the current epoch
  train_loss_batchSize1 = train_epoch_batchSize1(train_loader_batchSize1, network, optimizer, criterion, hparams_batchSize1)
  train_losses_batchSize1.append(train_loss_batchSize1)

  # TODO: Compute & save the average test loss & accuracy for the current epoch
  # TIP: Review the functions previously defined to implement the train/test epochs 
  test_loss_batchSize1,test_accuracy_batchSize1=test_epoch_batchSize1(test_loader_batchSize1, network, hparams_batchSize1)
  test_losses_batchSize1.append(test_loss_batchSize1)
  test_accs_batchSize1.append(test_accuracy_batchSize1)

  temps_batchSize1 = (time.time() - start_time3)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.000002
Train Epoch: 1 [100/60000 (0%)]	Loss: 0.000000


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Epoch: 1 [200/60000 (0%)]	Loss: 0.022612
Train Epoch: 1 [300/60000 (0%)]	Loss: 0.000000
Train Epoch: 1 [400/60000 (1%)]	Loss: 0.000000
Train Epoch: 1 [500/60000 (1%)]	Loss: 0.000001
Train Epoch: 1 [600/60000 (1%)]	Loss: 0.000000
Train Epoch: 1 [700/60000 (1%)]	Loss: 0.000021
Train Epoch: 1 [800/60000 (1%)]	Loss: 0.002538
Train Epoch: 1 [900/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1000/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1100/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1200/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1300/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1400/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1500/60000 (2%)]	Loss: 0.000000
Train Epoch: 1 [1600/60000 (3%)]	Loss: 0.000000
Train Epoch: 1 [1700/60000 (3%)]	Loss: 0.000000
Train Epoch: 1 [1800/60000 (3%)]	Loss: 4.613550
Train Epoch: 1 [1900/60000 (3%)]	Loss: 0.000000
Train Epoch: 1 [2000/60000 (3%)]	Loss: 0.000006
Train Epoch: 1 [2100/60000 (4%)]	Loss: 0.000000
Train Epoch: 1 [2200/60000 (4%)]	Loss: 0.000000


In [ ]:
import datetime
print('Per a batch_size = 64, inicialització random, tenim ' + str(datetime.timedelta(seconds=temps_batchSize64)))
print('Per a batch_size = 64, inicialització a Zeros, tenim ' + str(datetime.timedelta(seconds=temps_batchSize64_netZeros)))
print('Per a batch_size = 64, inicialització a Xavier, tenim ' + str(datetime.timedelta(seconds=temps_batchSize64_xavier)))
print()
print('Per a batch_size = 1000, inicialització random, tenim ' + str(datetime.timedelta(seconds=temps_batchSize1000)))
print()
print('Per a batch_size = 500, inicialització random, tenim ' + str(datetime.timedelta(seconds=temps_batchSize500)))
print('Per a batch_size = 500, inicialització a Xavier, tenim ' + str(datetime.timedelta(seconds=temps_batchSize64_xavier_500)))
print('Per a batch_size = 500, inicialització a Zeros, tenim ' + str(datetime.timedelta(seconds=temps_batchSize64_netZeros_500)))
print()
print('Per a batch_size = 1, i només una època, tenim ' + str(datetime.timedelta(seconds=temps_batchSize1)))
print('    Multiplicant per a 10 èpoques, equival a ' + str(datetime.timedelta(seconds=10*temps_batchSize1)))


In [ ]:
plt.figure()

data =('Batch Size = 64, random', 'Batch Size = 64, zeros', 'Batch Size = 64, xavier')
pos = np.arange(len(data))
temps = [temps_batchSize64, temps_batchSize64_netZeros, temps_batchSize64_xavier]

# change the bar color to be less bright blue
bars = plt.bar(pos, temps, align='center', linewidth=0, color='lightslategrey')
# make one bar, the python bar, a contrasting color
bars[2].set_color('#1F77B4')

# soften all labels by turning grey
plt.xticks(pos, data, alpha=0.8, rotation = 45)
# remove the Y label since bars are directly labeled
#plt.ylabel('% Popularity', alpha=0.8)
plt.title('Execution times when batch size = 64 for MNIST train + test', alpha=.9)

# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

# remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)
    
# direct label each bar with Y axis values
for bar in bars:
    plt.gca().text(bar.get_x() + bar.get_width()/2, bar.get_height() - 5, str(int(bar.get_height())) + ' sec', 
                 ha='center', color='w', fontsize=11)
plt.show()

In [ ]:
plt.figure()

data = ('Batch Size = 500, random', 'Batch Size = 500, zeros', 'Batch Size = 500, xavier')
pos = np.arange(len(data))
temps = [temps_batchSize500, temps_batchSize64_netZeros_500, temps_batchSize64_xavier_500]

# change the bar color to be less bright blue
bars = plt.bar(pos, temps, align='center', linewidth=0, color='lightslategrey')
# make one bar, the python bar, a contrasting color
bars[2].set_color('#1F77B4')

# soften all labels by turning grey
plt.xticks(pos, data, alpha=0.8, rotation = 45)
# remove the Y label since bars are directly labeled
#plt.ylabel('% Popularity', alpha=0.8)
plt.title('Execution times when batch size = 500 for MNIST train + test', alpha=0.8)

# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

# remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)
    
# direct label each bar with Y axis values
for bar in bars:
    plt.gca().text(bar.get_x() + bar.get_width()/2, bar.get_height() - 5, str(int(bar.get_height())) + ' sec', 
                 ha='center', color='w', fontsize=11)

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(2,2,1)
plt.title('batch_size = 64', fontsize=15)
plt.xlabel('Epoch')
plt.ylim(0,0.2)
plt.ylabel('NLLLoss')
plt.plot(train_losses, label='train')
plt.plot(test_losses, label='test')
plt.legend()

plt.subplot(2,2,3)
plt.title('batch_size = 64', fontsize=15)
plt.ylim(95,100)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs)

plt.subplot(2,2,2)
plt.title('batch_size = 1000', fontsize=15)
plt.ylim(0,0.2)
plt.xlabel('Epoch')
plt.ylabel('NLLLoss')
plt.plot(train_losses_batchSize1000, label='train')
plt.plot(test_losses_batchSize1000, label='test')
plt.legend()

plt.subplot(2,2,4)
plt.title('batch_size = 1000', fontsize=15)
plt.ylim(95,100)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs_batchSize1000)

espai = 0.3
plt.subplots_adjust(hspace=espai, wspace=espai)

## Plots d'inicialitzacions de pesos

In [ ]:
plt.figure(figsize=(26,6))

plt.subplot(1,3,1)
plt.title('batch_size = 64, inicialització random', fontsize=15)
plt.xlabel('Epoch')
plt.ylim(85,100)
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs)

plt.subplot(1,3,2)
plt.title('batch_size = 64, inicialització a Xavier', fontsize=15)
plt.xlabel('Epoch')
plt.ylim(85,100)
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs_xavier)

plt.subplot(1,3,3)
plt.text(3, 14, s="L'eix de les y és diferent! 10-15%.", bbox=dict(facecolor='red', alpha=0.5), fontsize=15)
plt.title('batch_size = 64, inicialització a Zeros', fontsize=15)
plt.ylim(10,15)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs_zeros)

espai = 0.3
plt.subplots_adjust(hspace=espai, wspace=espai)

In [ ]:
plt.figure(figsize=(30,6))
plt.subplot(1,3,1)
plt.title('batch_size = 500, inicialització random', fontsize=15)
plt.ylim(85,100)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs_500)

plt.subplot(1,3,2)
plt.title('batch_size = 500, inicialització a Xavier', fontsize=15)
plt.ylim(85,100)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs_xavier_500)


plt.subplot(1,3,3)
plt.text(3, 14, s="L'eix de les y és diferent! 10-15%.", bbox=dict(facecolor='red', alpha=0.5), fontsize=15)
plt.title('batch_size = 500, inicialització a Zeros', fontsize=15)
plt.xlabel('Epoch')
plt.ylim(10,15)
plt.ylabel('Test Accuracy [%]')
plt.plot(test_accs_zeros_500)

espai = 0.3
plt.subplots_adjust(hspace=espai, wspace=espai)